In [2]:
import pandas as pd

In [3]:
df_transaction = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv', parse_dates=['de_dt'])
df_product = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv')
df_action = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv')

/root/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_transaction.tail(1)

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
599960,35311,2839,6,B03,0324,2019-09-27,14:09,6800,1


#### The Goal
we will build collaborative filtering models for recommending products to customers using purchase data. 
In particular, we'll cover in details the step-by-step process in constructing a recommendation system with Python and machine learning module Turicreate. These steps include:
- Transforming and normalizing data
- Training models
- Evaluating model performance
- Selecting the optimal model

When a customer first taps oh the "order" page, we may recommend top 10 items to be added to their basket, e.g. disposable utensils, fresh meat, chips, and and so on.

- Input : customer ID
- Returns : ranked list of items(product IDs), that the user is most likely to want to put in his/her(empty) "basket"

### Implementation
1. Import modules
- `pandas` and `numpy` for data manipulation
- `turicreate` for performing model selection and evaluation
- `sklearn` for splitting the data into train and test set

In [41]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

pd.options.display.max_rows = 100

### 1. Create data with user, item and target field

In [6]:
transaction = df_transaction[['clnt_id', 'pd_c', 'buy_ct']]

In [9]:
transaction.sort_values(by='clnt_id', inplace=True)

In [12]:
transaction.tail()

,clnt_id,pd_c,buy_ct
17665,72424,0957,1
17775,72424,0354,1
17843,72424,0178,1
13315,72424,1395,1
10963,72424,1617,1


In [20]:
data = pd.melt(transaction.set_index('clnt_id')['pd_c'].apply(pd.Series).reset_index(), id_vars=['clnt_id'], value_name='product')

In [34]:
data = data.dropna().drop(['variable'], axis=1).groupby(['clnt_id', 'pd_c']).agg({'pd_c':'count'}).rename(columns={'pd_c':'purchase_count'})\
    .reset_index()

In [35]:
data.loc[data['pd_c'] == 'unknown', 'pd_c'] = 0

In [36]:
data['pd_c'] = data['pd_c'].astype(np.int64)

In [42]:
print(data.shape)
data.head(100)

(332184, 3)


,clnt_id,pd_c,purchase_count
0,2,92,3
1,2,151,1
2,2,186,1
3,2,189,1
4,2,351,1
5,2,857,1
6,2,1015,1
7,2,1395,3
8,9,114,1
9,9,188,1


### 2. Create dummy
- Dummy for marking whether a customer bought that item or not.
- If one buys an item, then purchase_dummy are marked as 1

In [43]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

data_dummy = create_data_dummy(data)

### 3. Nomalize item values across users
- To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [46]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='clnt_id', columns='pd_c')

In [47]:
df_matrix

pd_c,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667
clnt_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72410,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N

In [48]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [49]:
df_matrix_norm

pd_c,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667
clnt_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0.047619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,0.011905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72410,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [52]:
# create a table for input to the modeling
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['clnt_id'], value_name='scaled_purchase_freq').dropna()

print(data_norm.shape)
data_norm.head()

(330811, 3)


,clnt_id,pd_c,scaled_purchase_freq
2,12,0,0.047619
3,20,0,0.011905
5,24,0,0.000000
8,40,0,0.000000
9,41,0,0.000000


In [53]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='clnt_id', columns='pd_c')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index_names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['clnt_id'], value_name='scaled_purchase_freq').dropna()

### 4. Split train and test set

In [58]:
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [59]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

### 5. Define Models using Turicreate library

In [60]:
# constant variables to define field names include:
user_id = 'clnt_id'
item_id = 'pd_c'
users_to_recommend = list(transaction['clnt_id'])
n_rec = 10 #number of items to recommend
n_display = 30 # to display the first few rows 

In [74]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model 

### 6. Popularity Model as Baseline
- The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
- Training data is used for model selection

*i. Using purchase count*

In [63]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 265747 observations with 11008 users and 1640 items.

Data prepared in: 0.288205s

265747 observations to process; with 1640 unique items.

recommendations finished on 1000/599961 queries. users per second: 108378

...

recommendations finished on 596000/599961 queries. users per second: 153044

recommendations finished on 597000/599961 queries. users per second: 152823

recommendations finished on 598000/599961 queries. users per second: 152518

recommendations finished on 599000/599961 queries. users per second: 152249

+---------+------+--------------------+------+
| clnt_id | pd_c |       score        | rank |
+---------+------+--------------------+------+
|    2    | 1038 | 10.590909090909092 |  1   |
|    2    | 193  |        7.0         |  2   |
|    2    | 904  |        5.0         |  3   |
|    2    | 637  |        4.5         |  4   |
|    2    |  33  | 4.491428571428571  |  5   |
|    2    | 1275 |        4.0         |  6   |
|    2    |  0   | 3.971191135734072  |  7   |
|    2    | 1213 | 3.717345399698341  |  8   |
|    2    | 619  | 3.6666666666666665 |  9   |
|    2    | 526  | 3.654690618762475  |  10  |
|    2    | 1038 | 10.590909090909092 |  1   |
|    2    | 193  |        7.0         |  2   |
|    2    | 904  |        5.0         |  3   |
|    2    | 637  |        4.5         |  4   |
|    2    |  33  | 4.491428571428571  |  5   |
|    2    | 1275 |        4.0         |  6   |
|    2    |  0   | 3.971191135734072  |  7   |
|    2    | 1213 | 3.717345399698341  |  8   |
|    2    | 6

*ii. Using purchase dummy*

In [64]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 265747 observations with 11008 users and 1639 items.

Data prepared in: 0.187397s

...

recommendations finished on 596000/599961 queries. users per second: 168775

recommendations finished on 597000/599961 queries. users per second: 168563

recommendations finished on 598000/599961 queries. users per second: 168205

recommendations finished on 599000/599961 queries. users per second: 167768

+---------+------+-------+------+
| clnt_id | pd_c | score | rank |
+---------+------+-------+------+
|    2    | 198  |  1.0  |  1   |
|    2    | 290  |  1.0  |  2   |
|    2    | 917  |  1.0  |  3   |
|    2    | 110  |  1.0  |  4   |
|    2    | 182  |  1.0  |  5   |
|    2    | 1224 |  1.0  |  6   |
|    2    | 1184 |  1.0  |  7   |
|    2    | 274  |  1.0  |  8   |
|    2    | 1189 |  1.0  |  9   |
|    2    | 369  |  1.0  |  10  |
|    2    | 198  |  1.0  |  1   |
|    2    | 290  |  1.0  |  2   |
|    2    | 917  |  1.0  |  3   |
|    2    | 110  |  1.0  |  4   |
|    2    | 182  |  1.0  |  5   |
|    2    | 1224 |  1.0  |  6   |
|    2    | 1184 |  1.0  |  7   |
|    2    | 274  |  1.0  |  8   |
|    2    | 1189 |  1.0  |  9   |
|    2    | 369  |  1.0  |  10  |
|    2    | 198  |  1.0  |  1   |
|    2    | 290  |  1.0  |  2   |
|    2    | 917  |  1.0  |  3   |
|    2    | 110  |  1.0  |  4   |
|    2    | 182  |  1.0  |  5   |
|    2    | 1224 |  1.0  |  6   |
|    2    | 11

*iii. Using scaled purchase count*

In [65]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 264648 observations with 10950 users and 1328 items.

Data prepared in: 0.176784s

264648 observations to process; with 1328 unique items.

recommendations finished on 1000/599961 queries. users per second: 223714

...

recommendations finished on 596000/599961 queries. users per second: 190741

recommendations finished on 597000/599961 queries. users per second: 190707

recommendations finished on 598000/599961 queries. users per second: 190694

recommendations finished on 599000/599961 queries. users per second: 190651

+---------+------+--------------------+------+
| clnt_id | pd_c |       score        | rank |
+---------+------+--------------------+------+
|    2    | 1296 |        1.0         |  1   |
|    2    | 285  |        1.0         |  2   |
|    2    | 622  |        1.0         |  3   |
|    2    | 1283 |        1.0         |  4   |
|    2    | 1018 |        1.0         |  5   |
|    2    | 757  |        1.0         |  6   |
|    2    | 1168 | 0.6666666666666666 |  7   |
|    2    | 508  | 0.6666666666666666 |  8   |
|    2    | 986  | 0.6666666666666666 |  9   |
|    2    | 1270 | 0.6666666666666666 |  10  |
|    2    | 1296 |        1.0         |  1   |
|    2    | 285  |        1.0         |  2   |
|    2    | 622  |        1.0         |  3   |
|    2    | 1283 |        1.0         |  4   |
|    2    | 1018 |        1.0         |  5   |
|    2    | 757  |        1.0         |  6   |
|    2    | 1168 | 0.6666666666666666 |  7   |
|    2    | 508  | 0.6666666666666666 |  8   |
|    2    | 9

*i. Using purchase count*

In [70]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 265747 observations with 11008 users and 1640 items.

Data prepared in: 0.348178s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 596000/599961 queries. users per second: 85536

recommendations finished on 597000/599961 queries. users per second: 85493.7

recommendations finished on 598000/599961 queries. users per second: 85441.5

recommendations finished on 599000/599961 queries. users per second: 85437.9

+---------+------+---------------------+------+
| clnt_id | pd_c |        score        | rank |
+---------+------+---------------------+------+
|    2    |  0   |  0.5712678943361554 |  1   |
|    2    | 1213 |  0.5505238856588092 |  2   |
|    2    | 347  | 0.48498839139938354 |  3   |
|    2    | 964  | 0.44743267978940693 |  4   |
|    2    | 1617 |  0.4389374256134033 |  5   |
|    2    | 1210 |  0.4144793578556606 |  6   |
|    2    | 194  | 0.41017087016786846 |  7   |
|    2    | 1393 |  0.4047508324895586 |  8   |
|    2    | 188  |  0.3971866709845407 |  9   |
|    2    | 864  | 0.38950384514672415 |  10  |
|    2    |  0   |  0.5712678943361554 |  1   |
|    2    | 1213 |  0.5505238856588092 |  2   |
|    2    | 347  | 0.48498839139938354 |  3   |
|    2    | 964  | 0.44743267978940693 |  4   |
|    2    | 1617 |  0.4389374256134033 |  5   |
|    2    | 1210 |  0.4144793578556606 |  6   |
|    2    | 194  | 0.41017087016786846 |  7   |
|    2    | 1393 |  0.4047508324895586 |

*ii. Using purchase dummy*

In [71]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 265747 observations with 11008 users and 1639 items.

Data prepared in: 0.224791s

...

recommendations finished on 596000/599961 queries. users per second: 92148.8

recommendations finished on 597000/599961 queries. users per second: 92075.8

recommendations finished on 598000/599961 queries. users per second: 91857.6

recommendations finished on 599000/599961 queries. users per second: 91571.3

+---------+------+---------------------+------+
| clnt_id | pd_c |        score        | rank |
+---------+------+---------------------+------+
|    2    | 964  |  0.2456876380102975 |  1   |
|    2    | 1213 | 0.24024686643055507 |  2   |
|    2    | 1617 | 0.23505172559193202 |  3   |
|    2    | 347  | 0.22940301043646677 |  4   |
|    2    |  0   | 0.22397700377873012 |  5   |
|    2    | 194  | 0.22073876006262644 |  6   |
|    2    | 1210 | 0.21932084219796316 |  7   |
|    2    | 1394 |  0.2174023219517299 |  8   |
|    2    | 188  | 0.21593486411230906 |  9   |
|    2    | 354  | 0.19680144105638778 |  10  |
|    2    | 964  |  0.2456876380102975 |  1   |
|    2    | 1213 | 0.24024686643055507 |  2   |
|    2    | 1617 | 0.23505172559193202 |  3   |
|    2    | 347  | 0.22940301043646677 |  4   |
|    2    |  0   | 0.22397700377873012 |  5   |
|    2    | 194  | 0.22073876006262644 |  6   |
|    2    | 1210 | 0.21932084219796316 |  7   |
|    2    | 1394 |  0.2174023219517299 |

*iii. Using scaled purchase count*

In [72]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 264648 observations with 10950 users and 1328 items.

Data prepared in: 0.145294s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 596000/599961 queries. users per second: 97801.6

recommendations finished on 597000/599961 queries. users per second: 97451.6

recommendations finished on 598000/599961 queries. users per second: 96885.9

recommendations finished on 599000/599961 queries. users per second: 96309.1

+---------+------+----------------------+------+
| clnt_id | pd_c |        score         | rank |
+---------+------+----------------------+------+
|    2    | 964  | 0.008884027600288391 |  1   |
|    2    | 1213 | 0.008827820420265198 |  2   |
|    2    | 1394 |  0.008214071393013   |  3   |
|    2    | 1393 | 0.008145719766616821 |  4   |
|    2    | 1183 | 0.007786378264427185 |  5   |
|    2    | 194  | 0.007629111409187317 |  6   |
|    2    | 1210 | 0.007379069924354553 |  7   |
|    2    | 188  | 0.007151797413825989 |  8   |
|    2    | 1215 | 0.006794467568397522 |  9   |
|    2    | 347  | 0.006634995341300964 |  10  |
|    2    | 964  | 0.008884027600288391 |  1   |
|    2    | 1213 | 0.008827820420265198 |  2   |
|    2    | 1394 |  0.008214071393013   |  3   |
|    2    | 1393 | 0.008145719766616821 |  4   |
|    2    | 1183 | 0.007786378264427185 |  5   |
|    2    | 194  | 0.007629111409187317 |  6   |
|    2    | 1210 | 0.007379069924354553 |  7   |
|    2    | 188  | 0

*i. Using purchase count*

In [75]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 265747 observations with 11008 users and 1640 items.

Data prepared in: 0.171346s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 596000/599961 queries. users per second: 81743.8

recommendations finished on 597000/599961 queries. users per second: 81762.5

recommendations finished on 598000/599961 queries. users per second: 81743

recommendations finished on 599000/599961 queries. users per second: 81653.3

+---------+------+--------------------+------+
| clnt_id | pd_c |       score        | rank |
+---------+------+--------------------+------+
|    2    | 1038 | 10.590909090909092 |  1   |
|    2    | 193  |        7.0         |  2   |
|    2    | 904  | 4.935765888009753  |  3   |
|    2    | 637  |        4.5         |  4   |
|    2    |  33  | 4.491428571428572  |  5   |
|    2    | 1275 |        4.0         |  6   |
|    2    |  0   | 3.971988696970858  |  7   |
|    2    | 1213 | 3.7195743664794114 |  8   |
|    2    | 619  | 3.666666666666667  |  9   |
|    2    | 526  | 3.654690618762481  |  10  |
|    2    | 1038 | 10.590909090909092 |  1   |
|    2    | 193  |        7.0         |  2   |
|    2    | 904  | 4.935765888009753  |  3   |
|    2    | 637  |        4.5         |  4   |
|    2    |  33  | 4.491428571428572  |  5   |
|    2    | 1275 |        4.0         |  6   |
|    2    |  0   | 3.971988696970858  |  7   |
|    2    | 1213 | 3.7195743664794114 |  8   |
|    2    | 6

*i. Using purchase count*

In [76]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 265747 observations with 11008 users and 1640 items.

Data prepared in: 0.137661s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 596000/599961 queries. users per second: 82244.6

recommendations finished on 597000/599961 queries. users per second: 82161.7

recommendations finished on 598000/599961 queries. users per second: 82057.8

recommendations finished on 599000/599961 queries. users per second: 81900.8

+---------+------+--------------------+------+
| clnt_id | pd_c |       score        | rank |
+---------+------+--------------------+------+
|    2    | 1038 | 10.59090909090909  |  1   |
|    2    | 193  |        7.0         |  2   |
|    2    | 904  | 4.935765888009753  |  3   |
|    2    | 637  |        4.5         |  4   |
|    2    |  33  |  4.49142857142857  |  5   |
|    2    | 1275 |        4.0         |  6   |
|    2    |  0   | 3.971988696970862  |  7   |
|    2    | 1213 | 3.7195743664794128 |  8   |
|    2    | 619  | 3.666666666666667  |  9   |
|    2    | 526  | 3.6546906187624737 |  10  |
|    2    | 1038 | 10.59090909090909  |  1   |
|    2    | 193  |        7.0         |  2   |
|    2    | 904  | 4.935765888009753  |  3   |
|    2    | 637  |        4.5         |  4   |
|    2    |  33  |  4.49142857142857  |  5   |
|    2    | 1275 |        4.0         |  6   |
|    2    |  0   | 3.971988696970862  |  7   |
|    2    | 1213 | 3.7195743664794128 |  8   |
|    2    | 6

*ii. Using purchase dummy*

In [77]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 265747 observations with 11008 users and 1639 items.

Data prepared in: 0.127329s

...

recommendations finished on 596000/599961 queries. users per second: 122908

recommendations finished on 597000/599961 queries. users per second: 122575

recommendations finished on 598000/599961 queries. users per second: 122247

recommendations finished on 599000/599961 queries. users per second: 121922

+---------+------+-------+------+
| clnt_id | pd_c | score | rank |
+---------+------+-------+------+
|    2    | 198  |  0.0  |  1   |
|    2    | 290  |  0.0  |  2   |
|    2    | 917  |  0.0  |  3   |
|    2    | 110  |  0.0  |  4   |
|    2    | 182  |  0.0  |  5   |
|    2    | 1224 |  0.0  |  6   |
|    2    | 1184 |  0.0  |  7   |
|    2    | 274  |  0.0  |  8   |
|    2    | 1189 |  0.0  |  9   |
|    2    | 369  |  0.0  |  10  |
|    2    | 198  |  0.0  |  1   |
|    2    | 290  |  0.0  |  2   |
|    2    | 917  |  0.0  |  3   |
|    2    | 110  |  0.0  |  4   |
|    2    | 182  |  0.0  |  5   |
|    2    | 1224 |  0.0  |  6   |
|    2    | 1184 |  0.0  |  7   |
|    2    | 274  |  0.0  |  8   |
|    2    | 1189 |  0.0  |  9   |
|    2    | 369  |  0.0  |  10  |
|    2    | 198  |  0.0  |  1   |
|    2    | 290  |  0.0  |  2   |
|    2    | 917  |  0.0  |  3   |
|    2    | 110  |  0.0  |  4   |
|    2    | 182  |  0.0  |  5   |
|    2    | 1224 |  0.0  |  6   |
|    2    | 11

*iii. Using scaled purchase count*

In [78]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 264648 observations with 10950 users and 1328 items.

Data prepared in: 0.149868s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 596000/599961 queries. users per second: 86078.9

recommendations finished on 597000/599961 queries. users per second: 85769.2

recommendations finished on 598000/599961 queries. users per second: 85256.5

recommendations finished on 599000/599961 queries. users per second: 84756.8

+---------+------+--------------------+------+
| clnt_id | pd_c |       score        | rank |
+---------+------+--------------------+------+
|    2    | 1296 |        1.0         |  1   |
|    2    | 285  |        1.0         |  2   |
|    2    | 1018 |        1.0         |  3   |
|    2    | 622  |        1.0         |  4   |
|    2    | 1283 |        1.0         |  5   |
|    2    | 757  |        1.0         |  6   |
|    2    | 1168 | 0.6666666666666667 |  7   |
|    2    | 1270 | 0.6666666666666666 |  8   |
|    2    | 508  | 0.6666666666666666 |  9   |
|    2    | 986  | 0.6666666666666666 |  10  |
|    2    | 1296 |        1.0         |  1   |
|    2    | 285  |        1.0         |  2   |
|    2    | 1018 |        1.0         |  3   |
|    2    | 622  |        1.0         |  4   |
|    2    | 1283 |        1.0         |  5   |
|    2    | 757  |        1.0         |  6   |
|    2    | 1168 | 0.6666666666666667 |  7   |
|    2    | 1270 | 0.6666666666666666 |  8   |
|    2    | 5

In [80]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [81]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/9073 queries. users per second: 24329.1

recommendations finished on 2000/9073 queries. users per second: 40020.8

recommendations finished on 3000/9073 queries. users per second: 52355.1

recommendations finished on 4000/9073 queries. users per second: 61183.6

...

recommendations finished on 6000/9091 queries. users per second: 69719.6

recommendations finished on 7000/9091 queries. users per second: 71522

recommendations finished on 8000/9091 queries. users per second: 72432.9

recommendations finished on 9000/9091 queries. users per second: 68801.6


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 0.0002199978000219993  | 0.00011523694286866656 |
|   2    | 0.00016499835001649972 | 0.00022523584287966633 |
|   3    | 0.0001466652000146663  | 0.00023309290716616644 |
|   4    | 0.00010999890001099992 | 0.00023309290716616628 |
|   5    | 0.00010999890001099984 | 0.0002488070357391666  |
|   6    | 0.00010999890001099992 | 0.00025527755926922554 |
|   7    | 0.00010999890001099992 | 0.00025817226716425137 |
|   8    |  9.62490375096249e-05  | 0.00025817226716425153 |
|   9    |  8.55547000085555e-05  | 0.0002581722671642514  |
|   10   | 8.799912000879986e-05  | 0.00026817216716525175 |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13038810976766071

Per User RMSE (best)
+---------+----

### Evaluation Summary
- Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
- Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
- RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.

Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

### Final Output

In [87]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='scaled_purchase_freq', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 330811 observations with 11247 users and 1329 items.

Data prepared in: 0.161027s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 596000/599961 queries. users per second: 88057.5

recommendations finished on 597000/599961 queries. users per second: 87977.2

recommendations finished on 598000/599961 queries. users per second: 87891.1

recommendations finished on 599000/599961 queries. users per second: 87741.3

+---------+------+-----------------------+------+
| clnt_id | pd_c |         score         | rank |
+---------+------+-----------------------+------+
|    2    | 964  |  0.005744665861129761 |  1   |
|    2    | 1213 |  0.005573585629463196 |  2   |
|    2    | 1394 |  0.005449317395687103 |  3   |
|    2    | 1617 |  0.004879213869571686 |  4   |
|    2    | 1393 |  0.004811331629753113 |  5   |
|    2    | 1183 |  0.004782430827617645 |  6   |
|    2    | 194  |  0.004738248884677887 |  7   |
|    2    | 1210 |  0.004481777548789978 |  8   |
|    2    | 188  | 0.0044020116329193115 |  9   |
|    2    | 347  |  0.004219621419906616 |  10  |
|    2    | 964  |  0.005744665861129761 |  1   |
|    2    | 1213 |  0.005573585629463196 |  2   |
|    2    | 1394 |  0.005449317395687103 |  3   |
|    2    | 1617 |  0.004879213869571686 |  4   |
|    2    | 1393 |  0.004811331629753113 |  5   |
|    2    | 1183 |  0.004782430827617645 |  6   |
|    2    | 194  |  0.004738248884677887 |  7   |


In [89]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(5999610, 4)


,clnt_id,pd_c,score,rank
0,2,964,0.005745,1
1,2,1213,0.005574,2
2,2,1394,0.005449,3
3,2,1617,0.004879,4
4,2,1393,0.004811,5


In [92]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['clnt_id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('clnt_id').set_index('clnt_id')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [94]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/599961 queries. users per second: 84118.4

recommendations finished on 2000/599961 queries. users per second: 87454.6

recommendations finished on 3000/599961 queries. users per second: 86695.2

recommendations finished on 4000/599961 queries. users per second: 90503.9

recommendations finished on 5000/599961 queries. users per second: 92891.9

...

recommendations finished on 597000/599961 queries. users per second: 88227.5

recommendations finished on 598000/599961 queries. users per second: 87860.8

recommendations finished on 599000/599961 queries. users per second: 87290.2

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(11284, 1)


,recommendedProducts
clnt_id,
2,1296|285|1018|622|1283|757|1168|1270|508|986|1...
9,1296|285|1018|622|1283|757|1168|1270|508|986|1...
12,1296|285|1018|622|1283|757|1168|1270|508|986|1...
20,1296|285|1018|622|1283|757|1168|1270|508|986|1...
23,1296|285|1018|622|1283|757|1168|1270|508|986|1...


### Customer recommendation function

In [101]:
def customer_recomendation(clnt_id):
    if clnt_id not in df_output.index:
        print('Customer not found.')
        return clnt_id
    return df_output.loc['clnt_id']

In [102]:
customer_recomendation(21)

Customer not found.


21

In [100]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11284 entries, 2 to 72424
Data columns (total 1 columns):
recommendedProducts    11284 non-null object
dtypes: object(1)
memory usage: 496.3+ KB


In [106]:
df_output['clnt_id']

KeyError: 'clnt_id'